In [1]:
%matplotlib inline
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import seaborn as sns
import seaborn.objects as so

In [2]:
df = pd.read_csv('data/info_voiture.csv')

In [3]:
df.columns

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')

In [4]:
with open('info_cols.txt', 'w') as f:
    for col in df.columns:
        unique_values = getattr(df, col).unique()
        f.write(f"Colonnes {col} : {unique_values}\n")


In [5]:
for col in df.columns:
    unique_values = getattr(df, col).unique()
    print(f"Colonnes {col} : {unique_values}")

Colonnes car_ID : [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205]
Colonnes symboling : [ 3  1  2  0 -1 -2]
Colonnes CarName : ['alfa-romero giulia' 'alfa-romero stelvio' 'alfa-romero Quadrifoglio'
 'audi 100 ls' 'au

### <p style="color:grey">🗒 Suppression colonne ID </p>

In [6]:
# suppression de la colonne car_ID
df = df.drop('car_ID', axis=1)


### <p style="color:grey">🗒 Séparation marque et modele </p>

In [7]:
# Ajouter les colonnes 'voiture' et 'modele' en divisant la colonne 'CarName'
df[['marque', 'modele']] = df['CarName'].str.split(' ', n=1, expand=True)

# Supprimer la colonne 'CarName'
df = df.drop('CarName', axis=1)

# Afficher les 10 premières lignes du DataFrame
display((df.head(5)))


,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero,giulia
1,3,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero,stelvio
2,1,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,...,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero,Quadrifoglio
3,2,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,...,3.19,3.40,10.0,102,5500,24,30,13950.0,audi,100 ls
4,2,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,...,3.19,3.40,8.0,115,5500,18,22,17450.0,audi,100ls


### <p style="color:grey">🗒 Traduction </p>

In [8]:
def translate_carbody(x):
    translations = {
        'convertible': 'décapotable',
        'hardtop': 'coupé',
        'hatchback': 'à hayon',
        'sedan': 'berline',
        'wagon': 'break'
    }
    return translations[x]

df['carbody'] = df['carbody'].apply(lambda x: translate_carbody(x))

display(df.head(2))

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,gas,std,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero,giulia
1,3,gas,std,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero,stelvio


In [9]:
def translate_drivewheel(x):
    translations = {
        'rwd': 'propulsion arrière',
        'fwd': 'traction avant',
        '4wd': '4 roues motrices'
    }
    return translations[x]

In [10]:
# Convertir la colonne 'price' du dollar à l'euro
df['price'] = round(df['price'] * 0.85, 2)
display(df.head(2))

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,gas,std,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,11470.75,alfa-romero,giulia
1,3,gas,std,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,14025.00,alfa-romero,stelvio


In [11]:
# Créer un dictionnaire pour mapper les valeurs à traduire
mapping = {'gas': 'essence', 'diesel': 'gasoil'}

# Utiliser la méthode replace() pour remplacer les valeurs dans la colonne 'fueltype'
df['fueltype'] = df['fueltype'].replace(mapping)

# Afficher les 10 premières lignes du dataset pour vérification
display(df.head(2))


,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,essence,std,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,11470.75,alfa-romero,giulia
1,3,essence,std,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,14025.00,alfa-romero,stelvio


In [12]:
df['aspiration'] = df['aspiration'].replace({'std': 'standard', 'turbo': 'turbocharged'})
display(df.head(1))

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,essence,standard,two,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,11470.75,alfa-romero,giulia


In [13]:
# Traduction
mapping = {
    'two': 'deux',
    'three': 'trois',
    'four': 'quatre',
    'five': 'cinq',
    'six': 'six',
    'eight': 'huit',
    'twelve': 'douze',
}

df['doornumber'] = df['doornumber'].map(mapping)
df['cylindernumber'] = df['cylindernumber'].map(mapping)


display(df.head(1))

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,essence,standard,deux,décapotable,rwd,front,88.6,168.8,64.1,...,3.47,2.68,9.0,111,5000,21,27,11470.75,alfa-romero,giulia


In [14]:
df.marque.unique()
# len(df.voiture.unique())

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'maxda', 'mazda', 'buick', 'mercury',
       'mitsubishi', 'Nissan', 'nissan', 'peugeot', 'plymouth', 'porsche',
       'porcshce', 'renault', 'saab', 'subaru', 'toyota', 'toyouta',
       'vokswagen', 'volkswagen', 'vw', 'volvo'], dtype=object)

In [15]:
def normalize_marque(marque):
    marque = marque.lower()
    if re.match("p(.*)(r.*)?c(.*)h(.*)e", marque):
        return "porsche"
    elif marque in ["maxda", "mazda"]:
        return "mazda"
    elif marque in ["nissan", "Nissan"]:
        return "nissan"
    elif marque in ["toyota", "toyouta"]:
        return "toyota"
    elif marque in ["vokswagen", "volkswagen", "vw"]:
        return "volkswagen"
    elif marque == "alfa-romero":
        return "alfa-romeo"
    else:
        return marque

df['marque'] = df['marque'].apply(normalize_marque)

df.marque.unique()

array(['alfa-romeo', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [16]:
df.marque.unique()

array(['alfa-romeo', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

### <p style="color:grey">🗒 Conversion unités </p>

In [17]:
# Conversion des unités
df['wheelbase'] = df['wheelbase'] * 2.54
df['carlength'] = df['carlength'] * 2.54
df['carwidth'] = df['carwidth'] * 2.54
df['carheight'] = df['carheight'] * 2.54
# df['curbweight'] = df['curbweight'] * 0.453592

In [18]:
#On convertit de livres(lbs) en tonnes(T)
df.curbweight = df.curbweight.apply(lambda x: round(0.45359*x/1000),2)

#On convertit de pouces cubes(in³) en centimètres cubes(cm³)
df.enginesize = df.enginesize.apply(lambda x: round(16.3871*x))

#On convertit de pouces(in) en milimètres
df.boreratio = df.boreratio.apply(lambda x : round(25.4*x))

#On convertit de Miles per gallon(mpg) en litres au 100(L/100km)
df.citympg = df.citympg.apply(lambda x: round(x/235.214,3))
df.highwaympg = df.highwaympg.apply(lambda x: round(x/235.214,3))

In [19]:
display(df.head(50))

,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,...,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,marque,modele
0,3,essence,standard,deux,décapotable,rwd,front,225.044,428.752,162.814,...,88,2.68,9.00,111,5000,0.089,0.115,11470.75,alfa-romeo,giulia
1,3,essence,standard,deux,décapotable,rwd,front,225.044,428.752,162.814,...,88,2.68,9.00,111,5000,0.089,0.115,14025.00,alfa-romeo,stelvio
2,1,essence,standard,deux,à hayon,rwd,front,240.030,434.848,166.370,...,68,3.47,9.00,154,5000,0.081,0.111,14025.00,alfa-romeo,Quadrifoglio
3,2,essence,standard,quatre,berline,fwd,front,253.492,448.564,168.148,...,81,3.40,10.00,102,5500,0.102,0.128,11857.50,audi,100 ls
4,2,essence,standard,quatre,berline,4wd,front,252.476,448.564,168.656,...,81,3.40,8.00,115,5500,0.077,0.094,14832.50,audi,100ls
5,2,essence,standard,deux,berline,fwd,front,253.492,450.342,168.402,...,81,3.40,8.50,110,5500,0.081,0.106,12962.50,audi,fox
6,1,essence,standard,quatre,berline,fwd,front,268.732,489.458,181.356,...,81,3.40,8.50,110,5500,0.081,0.106,15053.50,audi,100ls
7,1,essence,standard,quatre,break,fwd,front,268.732,489.458,181.356,...,81,3.40,8.50,110,5500,0.081,0.106,16082.00,audi,5000
8,1,essence,turbocharged,quatre,berline,fwd,front,268.732,489.458,181.356,...,80,3.40,8.30,140,5500,0.072,0.085,20293.75,audi,4000
9,0,essence,turbocharged,deux,à hayon,4wd,front,252.730,452.628,172.466,...,80,3.40,7.00,160,5500,0.068,0.094,15180.29,audi,5000s (diesel)
